<a href="https://colab.research.google.com/github/MartinBelderBMC/Extractie-Analyse-omgevingsplannen/blob/main/Basisextractie_HTML_plannen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Omgeving opzetten

In [ ]:
import os, re, itertools, requests
import urllib.request

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from itertools import chain

from google.colab import drive

from io import BytesIO

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Voor hier de geprefereerde werkmap in

werkmap = ""

resultaten_map = os.path.join(werkmap, "Resultaten")

# Definitie functies voor extractie lijst voor parsing

In [ ]:
# @title
def extract_raw_html(url = str):

  """Deze functie extraheert de html-string van een specifieke url en
  encodeert deze in utf-8

  Inputs:
  - url: string met adres

  Output:
  - string van ruwe html"""

  response = urllib.request.urlopen(url)

  return response.read().decode('utf-8')

In [ ]:
# @title
def create_archive_link_from_IMRO(IMRO = str, base_link = str):

  """Deze functie maakt een een link aan naar het
  archief van een specifieke IMRO

  Input:
  - IMRO: string van het IMRO-plan
  - base_link: string van de link naar het basisarchief van de gemeente in kwestie""

  Output:
  - string van de link naar het archief van het IMRO"""

  return base_link + IMRO

In [ ]:
# @title
def extract_regel_htmls(archief_html = str,
                        IMRO = str,
                        link_regex = """<(?:A HREF|a href)=(?:'|")(.+?)(?:'|")(?:>|\s)"""):

  """Deze functie extraheert een lijst met htmls waarin regels te vinden zijn
  uit het archief van een specifieke IMRO

  Input:
  - archief_html: string van de html van het archief van het IMRO
  - IMRO: string van het IMRO-plan
  - link_regex = string met regex van links naar htmls waarin regels te vinden zijn


  Output:
  - lijst met regel-htmls

  """
  # Maak lijst met hrefs, zoals gextraheerd door regex
  href_list = re.findall(link_regex, archief_html)

  # Iteratie door lijst met regex: als prefix 'r_' in href voorkomt, gevolgd
  # door de IMRO-code, extraheer de link vanaf de eerste plaats vanaf rechts waar
  # '/' geovnden wordt
  match_list = [href[href.rfind("/") + 1:] for href
                in href_list
                if 'r_' + IMRO in href and href[-5:] == '.html']

  # Als er geen matches zijn, return lege lijst
  if len(match_list) == 0:

    return []

  else:

    return match_list

In [ ]:
# @title
def maak_dict_met_te_parsen_htmls_per_IMRO(parse_plans, base_link):

  """Deze functie neemt een lijst met de te parsen IMRO's en maakt
  een dictionary aan van alle htmls met regels per IMRO

  Input:
  - parse_plans: dict met parsing-informatie per plan

  Output:
  - extract_regels_dict: dict met per IMRO een lijst met de link naar de
    htmls die te extraheren regels bevatten"""

  extract_regels_dict = {}

  # Itereer door alle IMRO-codes
  for IMRO in [parse_plans[key]['Code'] for key in parse_plans.keys()]:


    # Maak een link naar het archief per IMRO
    full_link = create_archive_link_from_IMRO(IMRO, base_link)

    print('IMRO: ', full_link)

    # Extraheer het archief per IMRO
    raw_html = extract_raw_html(full_link)

    # Extraheer de htmls met regels
    regel_html_list = extract_regel_htmls(raw_html, IMRO)

    # Maak lijst aan met volledige link van elke regel_html per IMRO en schrijf
    # deze naar extract_regels_dict
    extract_regels_dict[IMRO] = [create_archive_link_from_IMRO(IMRO, base_link) + '/' + regel_html
                                 for regel_html in regel_html_list]

  return extract_regels_dict

# Aanmaken van extractielijst

## **Beschrijving werkwijze:**

**Stap 1: Het extraheren van de lijst met te parsen html bestanden**

---




We doorzoeken de index van manifestloacaties van ruimtelijkeplannen.nl. Dit archief bevat een sub-archief van ruimtelijke plannen per gemeente. De variabele "base_link" is de link naar het archief van een specifieke gemeente.


---




We zoeken vervolgens in het archief de specifieke plannen die zijn aangeleverd. De plannen hebben allemaal een 'IMRO'-code. De relevante IMRO-codes worden vooraf aangeleverd. Deze hebben we opgeslagen in het dictionary 'parse_plans'. In dit dictionary is de key de naam van het plan. De IMRO-code is te vinden in het onderliggende dictionary achter de key 'Code'



---


Het archief per IMRO is doorgaans in twee varianten beschikbaar: een archief met alle regels en hoofstukken in een enkele html, of een html per artikel. Daarom parsen we het archief of alle htmls die het prefix 'r_' hebben, gevolgd door de IMRO_code. Dit zijn de htmls die de te extraheren regels bevatten.



---


We voeren de functie "***maak_dict_met_te_parsen_htmls_per_IMRO***" uit om een dictionary met de links naar alle relevante htmls per IMRO aan te maken



In [ ]:
# Voer hier de te parsen plannen in in onderstaand format
parse_plans  = {'Naam HTML' {"Code" : "IMRO Code" }}

In [ ]:
# @title
parse_plans = {name: plan_dict for name, plan_dict in list(parse_plans.items())}

In [ ]:
# Voer hier de link naar het archief in
base_link =  ''

extract_regels_dict = maak_dict_met_te_parsen_htmls_per_IMRO(parse_plans, base_link)

IMRO:  https://www.amersfoort.nl/ro-online/NL.IMRO.0307.BP00070-0303


# Definitie van functies voor parsen van de structuur van HTML

In [ ]:
def extract_cell_markers_from_html(response_per_html = str):

  """Deze functie parset de ruwe html: er zijn twee typen markers in de html-string
  die van belang zijn: markers die aangeven dat er  een nieuwe cel begint en markers
  die aangeven dat een cel sluit. Elke instantie van een cel-sluiter hoort bij
  de meest recent gevonden cell opent. We extraheren daarmee twee lijsten, met voor
  beide type marker de locatie, de string van de marker en een variabele die aangeeft
  of de gevonden marker een cel-opener of en cel-sluiter is. Deze lijsten voegen we
  samen en sorteren we op de locatie van de marker

  Inputs:
  - ruwe html

  Outputs:
  - lijst met tuple (integer van locatie, string van marker, type marker)"""

  cell_openers = [(mt.end(), mt.group() ,'open') for mt in re.finditer(r'(<div.*?(?<!/)>)', response_per_html)]

  cell_closers = [(mt.start(), mt.group(), 'close') for mt in re.finditer(r'</div.*?>', response_per_html)]

  marker_tuples = cell_openers + cell_closers

  return sorted(marker_tuples, key = lambda x: x[0])

In [ ]:
def extract_metadata_from_cell_opener(opener_text = str, metadata_types = ["class", "div id"]):

  """Deze functie extraheert de metadata per cel indien deze beschikbaar is

  Inputs:
  - opener_text: string met de content van de cel-opener marker
  - metadata_types: lijst met typen metadata die geextraheerd moeten worden

  Outputs:
  - dict met metadata
  """

  # Lege 'default'  dictionary
  meta_extract = {metadata_type: None for metadata_type in metadata_types}

  # Itereer door de opgegeven metadata_types
  for metadata_type in metadata_types:

    # als het metadata-type in de tekst voorkomt
    if metadata_type in opener_text:

      # pak stuk tekst vanaf de plaats dat metadata-type wordt gevonden
      text_from_meta_marker = opener_text[opener_text.find(metadata_type):]

      # bepaal de locatie van de eerstvolgende 2 quotes
      quote_locs = [mt.start() for mt in re.finditer(r'"', text_from_meta_marker)][:2]

      # extraheer stuk tussen de quotes en schrijf weg naar dictionary
      meta_extract[metadata_type] = text_from_meta_marker[quote_locs[0] + 1: quote_locs[1]]

  return meta_extract

In [ ]:
def parse_cell_structure(sorted_marker_tuples = list, max_depth = 10):

  """Deze functie analyseert de celstructuur van een html, dit doet de functie
  door te itereren door de lijst met markers en aan de hand daarvan te bepalen
  1) welke cellen onderdeel zijn van welke andere cellen en
  2) op welke 'diepte' specifieke cellen vallen
  3) en waar mogelijk metadata per cel te extraheren

  De informatie over de diepte en de celstructuur wordt weergegeven door een lijst
  van getallen die een boomstructuur representeren. Van rechty naar links, vanaf
  het eerste niet-nul element geeft de reeks getallen de nummering van de node die
  de 'ouder'-node zijn van de node in kwestie

  Deze functie itereert over de gevonden cellen en bepaalt op welke diepte de cel
  valt en de hoeveelste cel op deze diepte het betreft.

  Daarnaast wordt in deze iteratie per celopener de beschikbare metadata geextraheerd

  Inputs:
  - sorted_marker_tuples: lijst van tuples, output van "extract_cell_markers_from_html"
  - max_depth: integer die aangeeft hoe diep de cellen kunnen zijn

  Outputs:
  - extracted_cells: dict met als key een een lijst

  """

  # Stel in: de startdiepte van waarop nodes worden geteld
  cell_level = 0

  # Dictionary om resultaten weg te schrijven
  extracted_cells = {}

  # Lege lijst om als 'geheugen'  van gevonden celopeners te dienen
  cell_openers = []

  # Bepaal de maximale lengte van de lijst van getallen oftewel maximale
  # boomdiepte, default diepte is 10
  cell_key = [0] * max_depth

  # Itereer door lijst met marker tuples
  for marker_pos, marker, marker_typ in sorted_marker_tuples:

    # Als de marker in kwestie een 'opener'-marker is:
    if marker_typ == 'open':
      # tel op de actuele diepte een node
      cell_key[cell_level] += 1

      # extraheer de metadata
      opener_metadata = extract_metadata_from_cell_opener(opener_text = marker, metadata_types = ["class", "div id"])

      # voeg de startnode toe aan het geheugen: positie, marker tekst, actuele cel-key, geextraheerde metadata
      cell_openers.append((marker_pos, marker, marker_typ, cell_key.copy(), opener_metadata))

      # verhoog de actuele diepte met 1
      cell_level += 1

    # Als de marker in kwestie een 'closer'-marker is
    elif marker_typ == 'close':

      # extraheer de corresponderende 'opener'-positie uit het laatste record in het 'geheugen'
      opener_pos = cell_openers[-1][0]

      # extraheer de key van de cel uit het geheugen
      extracted_cell_key = cell_openers[-1][3]

      # extraheer de metadata van de cel uit het geheugen
      cell_metadata = cell_openers[-1][4]

      # schrijf cel weg in dictionary
      extracted_cells[tuple(extracted_cell_key)] = {'startpositie cel': opener_pos, 'sluitpositie cel' : marker_pos} | cell_metadata

      # verwijder laatste record uit geheugen
      cell_openers = cell_openers[:-1]

      # zet teller voor het huidige niveau terug naar 0
      cell_key[cell_level] = 0

      # stel de actuele diepte voor celextractie 1 stap lager
      cell_level -= 1

  return extracted_cells

In [ ]:
def bepaal_unieke_content_per_cell(response_per_html: str, html_structure: dict):

  """Deze functie bepaalt per cel de unieke tekst, als in: de tekst per cel die
  *niet* ook in een 'kind' van de cel staat. Dit doen we door eerst de volledige
  inhoud per cel te extraheren op basis van de start- en sluitpositie van de cel
  """

  # Maak een dictionary aan met per cel een lege lijst
  content_per_cell = {cell_id:  response_per_html[cell_content['startpositie cel']: cell_content['sluitpositie cel']]
                      for cell_id, cell_content in html_structure.items()}

  # itereer door de geparsete structuur van de html
  for cell_id, cell_content in html_structure.items():

    # bepaal de 'parent' cell van elke cel
    cell_id_cutoff = len([val for val in cell_id if val > 0]) - 1

    # als uit de cell_id blijkt dat het de 'root' is, sla deze cel over aangezien
    # deze geen parent heeft
    if cell_id_cutoff == 0: continue

    parent_cell_id = cell_id[:cell_id_cutoff] + (0,) + cell_id[cell_id_cutoff + 1:]

    # bepaal het gedeelte van de html_string dat verwijderd mag worden van de parent
    # cell omdat het in een 'kind'-cel staat
    delete_slice = response_per_html[cell_content['startpositie cel']: cell_content['sluitpositie cel']]

    content_per_cell[parent_cell_id] = content_per_cell[parent_cell_id].replace(delete_slice, '')

  return content_per_cell

In [ ]:
def maak_key_per_cel(content_per_cell,
                     regex_key_extraction = r'<h[0-9].*>(.*)<\/h[0-9]>',
                     regex_key_level = r'[0-9](?:\.| )'):

  """Deze functie extraheert per cel de eventuele string die de key bevat en het niveau waarop deze zich bevindt"""

  # Lege dict aanmaken
  key_per_cell = {cell_id: None for cell_id in content_per_cell.keys()}

  # Iteratie door de tuple van cell id en content
  for cell_id, cell_content in content_per_cell.items():

    # Stel vast of de cel een key/titel bevat
    raw_key_extract = re.findall(regex_key_extraction, cell_content)

    if len(raw_key_extract) > 0:

      key = raw_key_extract[0]

      raw_key_level = re.findall(regex_key_level, key)

      key_per_cell[cell_id] = {'key':  key, 'level' : len(raw_key_level)}

    else:

      key_per_cell[cell_id] = {'key':  '', 'level' : -99}

  return key_per_cell

In [ ]:
def groepeer_cellen_onder_keys(content_per_cell, key_per_cell):

  grouped_cells = content_per_cell.copy()

  grouped_cells_keys = key_per_cell.copy()

  for cell_id, key_info in key_per_cell.items():

    # bepaal op welk punt de key moet worden aangepast om ofwel de 'parent' of de 'sibling' te vinden
    cell_id_cutoff = len([val for val in cell_id if val > 0]) - 1

    # bepaal de 'parent' cell: zelfde key, verander de laatste niet-nul waarde in de lijst naar nul
    parent_cell_id = cell_id[:cell_id_cutoff] + (0,) + cell_id[cell_id_cutoff + 1:]

    # bepaal de 'sibling' cell: zelfde key, trek 1 af bij de laatste niet-nul waarde in de lijst
    sibling_cell_id = cell_id[:cell_id_cutoff] + (cell_id[cell_id_cutoff] - 1,) + cell_id[cell_id_cutoff + 1:]

    # als de cel in kwestie geen key bevat
    if key_info['key'] == '':

      # check of de cel een 'sibling' heeft. ALs dit het geval is, voeg de content
      # toe aan sibling
      if sibling_cell_id in grouped_cells.keys():

        grouped_cells[sibling_cell_id] += grouped_cells[cell_id]

        grouped_cells.pop(cell_id)

        grouped_cells_keys.pop(cell_id)


      elif parent_cell_id in grouped_cells.keys():

        grouped_cells[parent_cell_id] += grouped_cells[cell_id]

        grouped_cells.pop(cell_id)

        grouped_cells_keys.pop(cell_id)

      else: continue

  return grouped_cells, grouped_cells_keys

In [ ]:
# @title
def bepaal_cellen_volledige_key(grouped_cells_keys):

  keys_in_html = set([key_info['level'] for key_info in grouped_cells_keys.values()])

  cell_ids_per_level = {level: [cell_id for cell_id in grouped_cells_keys.keys()
                                if grouped_cells_keys[cell_id]['level'] == level]
                        for level in keys_in_html}

  full_cell_ids = {cell_id: [cell_id] for cell_id in  grouped_cells_keys.keys()}

  for key_level in list(keys_in_html):

    for cell_id in cell_ids_per_level[key_level]:

      # bepaal op welk punt de key moet worden aangepast om ofwel de 'parent' of de 'sibling' te vinden
      cell_id_cutoff = len([val for val in cell_id if val > 0]) - 1

      # bepaal de 'parent' cell: zelfde key, verander de laatste niet-nul waarde in de lijst naar nul
      parent_cell_id = cell_id[:cell_id_cutoff] + (0,) + cell_id[cell_id_cutoff + 1:]

      if parent_cell_id in grouped_cells_keys.keys():

        full_cell_ids[cell_id] = full_cell_ids[parent_cell_id] + [cell_id]

  return full_cell_ids

In [ ]:
# @title
def voeg_key_bij_inhoud(grouped_cells, grouped_cells_keys,
                        full_cell_keys_per_grouped_cell):

  content_per_key = {}

  for cell_id in grouped_cells.keys():

    ids_full_key = full_cell_keys_per_grouped_cell[cell_id]

    full_key = [grouped_cells_keys[key_cell_id]['key'] for key_cell_id in ids_full_key]

    content_per_key[tuple(full_key)] = grouped_cells[cell_id]

  return content_per_key

In [ ]:
# @title
def opschonen_key_en_inhoud(content_per_key, regex_key_level = r'((?:[0-9]{1,2}(?:\.| )){1,3})'):

  clean_content_per_clean_key = {}

  for key, content in content_per_key.items():

    clean_key = []

    for key_item in key:

      if 'hoofdstuk' in key_item.lower(): continue

      delete_string = re.findall(regex_key_level, key_item)

      if len(delete_string) > 0:

        clean_key_item = key_item\
                          .replace(delete_string[0], '')\
                          .lower()\
                          .strip()\
                          .replace('\t', '')\
                          .replace('\n', '')\
                          .replace(':','')

        clean_key.append(clean_key_item)

      else:

        clean_key.append(key_item)

    if len(clean_key) > 1:

      clean_key = tuple(clean_key) + ('geen_key', ) * (3 - len(clean_key))

      clean_content = BeautifulSoup(content, 'lxml').text

      clean_content_per_clean_key[clean_key] = clean_content

  return clean_content_per_clean_key

In [ ]:
# @title
def test_result_per_IMRO(IMRO):

  test_result = pd.DataFrame(index = pd.MultiIndex.from_tuples(parse_dict_per_IMRO[IMRO].keys()),
                             data = parse_dict_per_IMRO[IMRO].values())

  return test_result

In [ ]:
# @title
def integriteitscheck_html(test_structure_frame, class_types = ['artikel', 'lid', 'sublid']):

  """Deze functie voert een check uit op de integriteit van de extractie per html.
  Het concept is als volgt:
  Het script gaat de vooraf geconfigureerde hiërarchie van typen  cel af (default is
  'artikel', 'lid', 'sublid', een hiërarchie met andere namen kan worden gespcecifieerd)
  en checkt of ieder sublid binnen een lid valt en ieder lid binnen een artikel.
  Bijgehouden wordt of er 'ongeplaatse' cellen van ieder type zijn. Daarnaast wordt ook
  geteld hoeveel verchillende cellen er zijn  van ieder type.

  Inputs:
  - test_structure_frame: Pandas dataframe met de gevonden cellen: de index is de
    celkey zoals bepaald door parse_cell_structure. De kolommen zijn: 'startpositie cel',
    'sluitpositie cel', 'class', 'div id';
  - class_types: list met strings van de benamingen van cellen, van links naar rechts
    hoger naar lager in de hiërarchie.

  Outputs:
  - dictionary met per html de tellingen van 'ongeplaatste' cellen per type en
    tellingen van per type cel.
  """

  # Bepaal welk type cellen door de integriteitscheck moet worden meegenomen
  parse_cells = test_structure_frame[test_structure_frame['class'].apply(lambda x: x in class_types)]

  # Maak subsets aan van cellen per type
  subsets = {class_type: test_structure_frame[test_structure_frame['class'] == class_type] for class_type in class_types}

  # Maak een lijst aan waarin de 'geplaatste' cq. 'gevalideerde cellen' kunnen worden weggeschreven
  placed_cells = []

  # Maak een dictionary aan om de aantallen per celtype te tellen
  count_class_types = {('aantal', class_type): 0 for class_type in class_types}

  # itereer door alle artikelaanduidingen
  for key_art, start_pos_art, end_pos_art, metadata_class, div_id_class in subsets['artikel'].itertuples(name = None, index = True):

    # plaats artikel in de lijst met gevalideerde cellen
    placed_cells.append(test_structure_frame.loc[key_art])

    count_class_types[('aantal','artikel')] += 1

    # itereer door alle lidaanduidingen
    for key_lid, start_pos_lid, end_pos_lid, metadata_lid, div_id_lid in subsets['lid'].itertuples(name = None):

      if start_pos_lid > start_pos_art and end_pos_lid < end_pos_art:

        placed_cells.append(test_structure_frame.loc[key_lid])

        count_class_types[('aantal','lid')] += 1

      # itereer door alle sublidaanduidingen
      for key_sublid, start_pos_sublid, end_pos_sublid, metadata_sublid, div_id_sublid in subsets['sublid'].itertuples(name = None):

        if start_pos_sublid > start_pos_lid and end_pos_sublid < end_pos_lid:

          placed_cells.append(test_structure_frame.loc[key_sublid])

          count_class_types[('aantal','sublid')] += 1

  # Als er cellen worden gevonden
  if len(placed_cells) > 0:

    placed_cells_frame = pd.concat(placed_cells, axis = 1).T

    failed_placements = parse_cells.drop(placed_cells_frame.index.values)

    failed_placement_count = {('geen plaatsing', class_type): failed_placements[failed_placements['class'] == class_type].shape[0] for class_type in class_types}


  else:

    failed_placement_count = {('geen plaatsing', class_type): 0 for class_type in class_types}

  out_dict = failed_placement_count | count_class_types

  return out_dict


# Parsing van HTML uitvoeren



## **Beschrijving werkwijze:** ##

We hebben een dictionary aangemaakt met daarin per IMRO een lijst met de verschillende links naar de htmls die bij bij deze specifieke IMRO horen.

---

We extraheren allereerst van iedere html de inhoud de inhoud schrijven we weg in het dictionary html_extracts. Als onderdeel van dit proces houden we een lijst bij van de htmls die niet geëxtraheerd kunnen worden

---

Vervolgens doorlopen we in een loop alle ruwe html. We doorlopen per html de volgende stappen:


1.   Het onderverdelen van de html in 'cellen' met inhoud. Normaliter staat een individueel artikel, lid of sublid om te extraheren in een afgebakende cel. We bepalen welke cellen er in de html staan met de functie **extract_cell_markers_from_html**
2.   De structuur van de html is zodanig dat de cellen een 'tree'-structuur hebben: een cel heeft één bovenliggende 'parent'-cell en meerdere onderliggende 'child'-cellen. Deze structuur stellen we vast met de functie **parse_cell_structure**



In [ ]:
# @title
failed_extract_files = []

html_extracts = {}

for IMRO, link_list in extract_regels_dict.items():

  html_extracts[IMRO] = {}

  for full_link in link_list:

    print(f'Ruwe HTML extraheren: {IMRO}: {full_link}')

    try:

      raw_extract = extract_raw_html(full_link)

    except:

      failed_extract_files.append((IMRO, full_link))

    html_extracts[IMRO][full_link] = raw_extract

Ruwe HTML extraheren: NL.IMRO.0307.BP00070-0303: https://www.amersfoort.nl/ro-online/NL.IMRO.0307.BP00070-0303/r_NL.IMRO.0307.BP00070-0303_2.10.html


In [ ]:
parse_dict_per_IMRO = {}

for IMRO in html_extracts.keys():

  print('Parsen: ', IMRO)

  parse_dict_per_IMRO[IMRO] = {}

  for link, response_per_html in html_extracts[IMRO].items():

    print('\t', link)

    sorted_marker_tuples = extract_cell_markers_from_html(response_per_html)

    html_structure = parse_cell_structure(sorted_marker_tuples)

    content_per_cell = bepaal_unieke_content_per_cell(response_per_html, html_structure)

    key_per_cell = maak_key_per_cel(content_per_cell)

    grouped_cells, grouped_cells_keys = groepeer_cellen_onder_keys(content_per_cell, key_per_cell)

    full_cell_keys_per_grouped_cell = bepaal_cellen_volledige_key(grouped_cells_keys)

    content_per_key = voeg_key_bij_inhoud(grouped_cells, grouped_cells_keys, full_cell_keys_per_grouped_cell)

    clean_content_per_clean_key = opschonen_key_en_inhoud(content_per_key)

    parse_dict_per_IMRO[IMRO].update(clean_content_per_clean_key)

Parsen:  NL.IMRO.0307.BP00070-0303
	 https://www.amersfoort.nl/ro-online/NL.IMRO.0307.BP00070-0303/r_NL.IMRO.0307.BP00070-0303_2.10.html
